In [95]:
import pandas as pd
import glob
import os
import re

In [96]:
#can change school, supported schools are: McGill, Georgetown, York, CMU, USC, AU, LIU
SCHOOL = "CMU"
keywords = ["China", "India", "Israel", "Palestine"]
data_path = f"journal_data/txt/{SCHOOL}"
output_path = f"bias_processing/data/1/{SCHOOL.lower()}_dataset.csv"
grouped_data_path = "grouped_data/csv"

In [97]:
# The output DataFrame
output_df = pd.DataFrame(columns=["date", "school", "keyword", "article"])

# Iterate through the keywords
for keyword in keywords:
    # Read the corresponding CSV file
    df = pd.read_csv(f"{grouped_data_path}/{SCHOOL}_{keyword}.csv")

    # Iterate through the dates in the DataFrame
    for date in df["date"]:
        # Format the date in the correct format for the .txt file name
        date_formatted = date.replace("-", "_")

        # Check if the .txt file exists
        txt_files = glob.glob(f"{data_path}/{date_formatted}*.txt")
        if txt_files:
            # Open the .txt file and read its content
            with open(txt_files[0], "r", encoding='utf-8', errors='ignore') as f:
                content = f.read()

            if(SCHOOL == "McGill"):
                # Split the content by the separator "The McGill Daily" repeated twice
                articles = content.split("The McGill Daily\nThe McGill Daily")

                # Iterate through the articles and check if the keyword is in the article
                for article in articles:
                    if keyword.lower() in article.lower():
                        # If the keyword is in the article, remove footer and create a new DataFrame and append it to the output DataFrame
                        cleaned_article = re.sub("Radio\nDownload file.*Powered by WordPress", "", article, flags=re.DOTALL)
                        new_row = pd.DataFrame({
                            "date": [date],
                            "school": [SCHOOL],
                            "keyword": [keyword],
                            "article": [cleaned_article.strip()]  # Remove leading/trailing white spaces
                        })
                        output_df = pd.concat([output_df, new_row], ignore_index=True)

            elif(SCHOOL == "Georgetown"):
                # Split the content by the separator "Your email address will not be published. Required fields are marked *"
                articles = content.split("Your email address will not be published. Required fields are marked *\n")

                # Iterate through the articles and check if the keyword is in the article
                for article in articles:
                    if keyword.lower() in article.lower():
                        # If the keyword is in the article, remove footer and create a new DataFrame and append it to the output DataFrame
                        cleaned_article = re.sub("Radio\nDownload file.*Powered by WordPress", "", article, flags=re.DOTALL)
                        new_row = pd.DataFrame({
                            "date": [date],
                            "school": [SCHOOL],
                            "keyword": [keyword],
                            "article": [cleaned_article.strip()]  # Remove leading/trailing white spaces
                        })
                        output_df = pd.concat([output_df, new_row], ignore_index=True)

            elif(SCHOOL == "York"):
                # Split the content by the separator "York University Community Newspaper"
                articles = content.split("York University Community Newspaper\n")

                # Iterate through the articles and check if the keyword is in the article
                for article in articles:
                    if keyword.lower() in article.lower():
                        # If the keyword is in the article, remove footer and create a new DataFrame and append it to the output DataFrame
                        cleaned_article = re.sub("Radio\nDownload file.*Powered by WordPress", "", article, flags=re.DOTALL)
                        new_row = pd.DataFrame({
                            "date": [date],
                            "school": [SCHOOL],
                            "keyword": [keyword],
                            "article": [cleaned_article.strip()]  # Remove leading/trailing white spaces
                        })
                        output_df = pd.concat([output_df, new_row], ignore_index=True)

            elif(SCHOOL == "CMU"):
                # Split the content by the separator "Carnegie Mellon's Student Newspaper Since 1906."
                articles = content.split("Carnegie Mellon's Student Newspaper Since 1906.\n")

                # Iterate through the articles and check if the keyword is in the article
                for article in articles:
                    if keyword.lower() in article.lower():
                        # If the keyword is in the article, remove footer and create a new DataFrame and append it to the output DataFrame
                        cleaned_article = re.sub("Radio\nDownload file.*Powered by WordPress", "", article, flags=re.DOTALL)
                        new_row = pd.DataFrame({
                            "date": [date],
                            "school": [SCHOOL],
                            "keyword": [keyword],
                            "article": [cleaned_article.strip()]  # Remove leading/trailing white spaces
                        })
                        output_df = pd.concat([output_df, new_row], ignore_index=True)

            elif(SCHOOL == "USC"):
            # Split the content by the separator "Extra en Español"
                articles = content.split("Extra en Español\n")

                # Iterate through the articles and check if the keyword is in the article
                for article in articles:
                    if keyword.lower() in article.lower():
                        # If the keyword is in the article, remove footer and create a new DataFrame and append it to the output DataFrame
                        cleaned_article = re.sub("Radio\nDownload file.*Powered by WordPress", "", article, flags=re.DOTALL)
                        new_row = pd.DataFrame({
                            "date": [date],
                            "school": [SCHOOL],
                            "keyword": [keyword],
                            "article": [cleaned_article.strip()]  # Remove leading/trailing white spaces
                        })
                        output_df = pd.concat([output_df, new_row], ignore_index=True)
            elif(SCHOOL == "AU"):
                # Split the content by the separator "Would you like to support our work? Donate here to The Eagle Innovation Fund."
                articles = content.split("Would you like to support our work? Donate here to The Eagle Innovation Fund.\n")

                # Iterate through the articles and check if the keyword is in the article
                for article in articles:
                    if keyword.lower() in article.lower():
                        # If the keyword is in the article, remove footer and create a new DataFrame and append it to the output DataFrame
                        cleaned_article = re.sub("Radio\nDownload file.*Powered by WordPress", "", article, flags=re.DOTALL)
                        new_row = pd.DataFrame({
                            "date": [date],
                            "school": [SCHOOL],
                            "keyword": [keyword],
                            "article": [cleaned_article.strip()]  # Remove leading/trailing white spaces
                        })
                        output_df = pd.concat([output_df, new_row], ignore_index=True)

            elif(SCHOOL == "LIU"):
                # Split the content by the separator "Your email address will not be published. Required fields are marked *" 
                articles = content.split("Your email address will not be published. Required fields are marked *.\n")

                # Iterate through the articles and check if the keyword is in the article
                for article in articles:
                    if keyword.lower() in article.lower():
                        # If the keyword is in the article, remove footer and create a new DataFrame and append it to the output DataFrame
                        cleaned_article = re.sub("Radio\nDownload file.*Powered by WordPress", "", article, flags=re.DOTALL)
                        new_row = pd.DataFrame({
                            "date": [date],
                            "school": [SCHOOL],
                            "keyword": [keyword],
                            "article": [cleaned_article.strip()]  # Remove leading/trailing white spaces
                        })
                        output_df = pd.concat([output_df, new_row], ignore_index=True)
# Add a line space between each article
output_df["article"] = output_df["article"] + "\n"

# Save the output DataFrame into a CSV file
output_df.to_csv(output_path, index=False)
print(output_df)

           date school    keyword  \
0    2009-01-19    CMU      China   
1    2009-02-09    CMU      China   
2    2009-02-23    CMU      China   
3    2009-02-23    CMU      China   
4    2009-02-23    CMU      China   
..          ...    ...        ...   
915  2023-02-27    CMU  Palestine   
916  2023-03-20    CMU  Palestine   
917  2023-04-10    CMU  Palestine   
918  2023-04-10    CMU  Palestine   
919  2023-04-24    CMU  Palestine   

                                               article  
0    Archives | \n          Join the Staff |\n     ...  
1    Archives | \n          Join the Staff |\n     ...  
2    Archives | \n          Join the Staff |\n     ...  
3    Archives | \n          Join the Staff |\n     ...  
4    Archives | \n          Join the Staff |\n     ...  
..                                                 ...  
915  Archives | \n          Join the Staff |\n     ...  
916  Archives | \n          Join the Staff |\n     ...  
917  Archives | \n          Join the Staff